In [ ]:
# download needed libraries
!pip3 install imageai --upgrade 
!pip install -q opencv-python


Requirement already up-to-date: imageai in /usr/local/lib/python3.7/dist-packages (2.1.6)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import the  libraries
from imageai.Detection.Custom import CustomObjectDetection, CustomVideoObjectDetection
from os import listdir
import smtplib
import time
import cv2

In [ ]:
# function used to send email when fire is found using the Simple Mail Transfer Protocol (SMTP)

def send_email():
    #the recieved email
    recipientEmail = "systemm.emaill@gmail.com" 
    smtp_server = "smtp.gmail.com"
    port = 587  # For starttls
    password = '*159951mmy'
    try:
      #the sender email
        #smtplib is Python’s built-in module for sending emails to any Internet machine 
        server = smtplib.SMTP(smtp_server, 587)
        server.ehlo()
        #Transport Layer Security
        server.starttls() 
        server.login("systemm.emaill@gmail.com", password)
        server.sendmail(recipientEmail,recipientEmail, "Warning A Fire Accident has been reported on ABC Company")
        print("sent to {}".format(recipientEmail))
        server.close()
    except Exception as e:
        print(e)

In [ ]:
#cut the video into small parts
def videoinput(cap):
  #path for the small videos
  shotsPath = '/content/drive/MyDrive/GP/v_input/f_%d.avi' # output path (must be avi, otherwize choose other codecs)
  #the number of frame in second
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  #the size of frame 
  size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
  #the codecc of video 
  # XVID codecs
  fourcc = int(cv2.VideoWriter_fourcc('X','V','I','D')) 
  idx=0 
  ret , frame = cap.read()# has frame returned
  while (cap.isOpened() and ret) :

      #video intervals (ex( 1 : 300))
      begen_Fidx = (idx * 300) + 1
      end_Fidx = (idx + 1) * 300       
      #write and save video in Path
      writer = cv2.VideoWriter(shotsPath%idx,fourcc,fps,size)
      cap.set(cv2.CAP_PROP_POS_FRAMES,begen_Fidx)
      print(writer.isOpened())
      while writer.isOpened():
        frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES) - 1
        if (frame_number < end_Fidx) and ret : 
          writer.write(frame)
        else:
          break
        ret, frame = cap.read()
      writer.release() 
      # increse idx by 2 to skip 300 frame
      idx += 2

In [ ]:
#extract images from video
def Frame_output(video_dict):
  #get video 
  cap1 = cv2.VideoCapture(video_dict)
  total_frame = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
  # save frame every # seconds
  seconds = 5
  fps = cap1.get(cv2.CAP_PROP_FPS) # Gets the frames per second
  # calculates number of frames that creates 10 seconds of video
  multiplier = fps * seconds
  # Check if camera opened successfully
  if (cap1.isOpened()== False):
      print("Error opening video stream or file")
  frame_counter = 1
  x=1
  ret, frame = cap1.read()
  while frame_counter <= total_frame and ret :
     
      x+=1

      #move from current possition to next possition
      cap1.set(cv2.CAP_PROP_POS_FRAMES, frame_counter)

      # save image
      cv2.imwrite("/content/drive/MyDrive/GP/input/{}.jpg".format(x),frame)

      #incress frame_counter by (multiplier = fps * seconds)
      frame_counter += multiplier
      ret, frame =cap1.read()


In [ ]:
#use YOLOv3 (pretrained model) from imageai library 
def detect_from_image(image_in,image_out):
    
    detector = CustomObjectDetection()
    detector.setModelTypeAsYOLOv3()
    # file path to trained model
    detector.setModelPath(detection_model_path="/content/drive/MyDrive/GP/finished/detection_model-ex-33--loss-4.97.h5")
    #set the file path to configuration json file
    detector.setJsonPath(configuration_json="/content/drive/MyDrive/GP/finished/detection_config.json")
    detector.loadModel()
    detections = detector.detectObjectsFromImage(input_image=image_in,
                                                 output_image_path=image_out,
                                                 minimum_percentage_probability=30)

    for detection in detections:
        print(detection["name"], " : ", detection["percentage_probability"])
    return detections 

In [ ]:
#calling functions
#cutting video to small parts and extract images
vidPath = '/content/drive/MyDrive/GP/videoes/video_final.mp4'
cap = cv2.VideoCapture(vidPath)
videoinput(cap)
videos_dir = '/content/drive/MyDrive/GP/v_input'
videos = listdir(videos_dir)
for video in videos:
  Frame_output(videos_dir+'/'+ video)
  cap.release()

True
True
True


In [ ]:
#calling fubctions
#detect fire and sending mails
output_dir = "/content/drive/MyDrive/GP/output/"
input_dir = "/content/drive/MyDrive/GP/input/"
image_input = listdir(input_dir)
dict_img = {image : image.split('.')[0] + "detected.jpg" for image in image_input}
x=0
for image_in,image_out in dict_img.items():
  print(image_in,image_out,"\n")
  val = detect_from_image(input_dir+image_in,output_dir+image_out)

  if (val != []) and (val[0]["name"]  == 'fire') and (x==0):
    send_email()
    x=1
  else:
    continue

6.jpg 6detected.jpg 



/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


7.jpg 7detected.jpg 

fire  :  36.82505786418915
sent to systemm.emaill@gmail.com
2.jpg 2detected.jpg 

3.jpg 3detected.jpg 

fire  :  36.82505786418915
